<a href="https://colab.research.google.com/github/RohaArslan/COMPUTERVISION/blob/main/transferlearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
from google.colab import drive
import zipfile
import os
from PIL import Image  # Correctly import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model

# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:

zip_path = '/content/drive/MyDrive/Dataset1.zip'
extract_path = '/content/Dataset1'
dataset_path = os.path.join(extract_path, 'train')

# Extract Dataset
if not os.path.exists(extract_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print(f"Dataset extracted to {extract_path}")
else:
    print(f"Dataset already exists at {extract_path}")


Dataset extracted to /content/Dataset1


In [31]:

# Function to clean dataset
def clean_dataset(dataset_path):
    for root, dirs, files in os.walk(dataset_path):
        for file in files:
            file_path = os.path.join(root, file)
            try:
                # Attempt to open the file as an image
                with Image.open(file_path) as img:
                    img.verify()  # Verify if it's a valid image
            except (IOError, SyntaxError):
                print(f"Removing invalid file: {file_path}")
                os.remove(file_path)

# Clean the dataset
clean_dataset('/content/Dataset1')

# Data Preparation
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

train_data = datagen.flow_from_directory(
    '/content/Dataset1',
    target_size=(224, 224),
    batch_size=32,
    subset="training"
)

val_data = datagen.flow_from_directory(
    '/content/Dataset1',
    target_size=(224, 224),
    batch_size=32,
    subset="validation"
)


Found 19999 images belonging to 1 classes.
Found 4999 images belonging to 1 classes.


In [33]:

# Load Pre-trained Model
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
base_model.trainable = False  # Freeze base model

# Custom Layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.2)(x)
output = Dense(train_data.num_classes, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=output)

# Compile and Train
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(train_data, validation_data=val_data, epochs=5)

# Save Model
model_save_path = "/content/drive/MyDrive/cats_dogs_transfer_learning_model.h5"
model.save(model_save_path)
print(f"Model saved to {model_save_path}")

Epoch 1/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 1509s 2s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 1516s 2s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 1475s 2s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 1548s 2s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 1513s 2s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00


Model saved to /content/drive/MyDrive/cats_dogs_transfer_learning_model.h5
